In [ ]:
#  pip install opencv-python

In [1]:
import cv2
import numpy as nwp

In [2]:
IMAGE_FILENAME = "map64.png"
img = cv2.imread(IMAGE_FILENAME)
IMAGE_SHAPE = img.shape[0:2]

In [3]:
if img is None:
  print("Error: %s cannot be read." % (IMAGE_FILENAME))
  quit()
else :
  print('Image Shape Dimensions : ' , IMAGE_SHAPE)

Image Shape Dimensions :  (64, 64)


In [4]:
def dec2bin(x, length):
    tmp = bin(x)[2:]
    for i in range(0, length - len(tmp)):
        tmp = "0" + tmp
    return tmp

Generating the bitmap for our field.
<br> Output file: **maps.sv** file storing our map field info in 3-bit values, encoded as follows: <br>
**100** : player<br>
**010** : exit<br>
**001** : wall<br>
**000** : free 

In [ ]:
def write(r,g,b,address,x,y): 
  r = str(int(r=='1111'))
  g = str(int(g=='1111'))
  b = str(int(b=='1111'))
  string = (("mem[%s] = 3'b" % address) + r + "" + g + "" + b + ";" + '\n')
  return string

def encoding(filename) :
  addr = 0
  fo = open(filename, "w+")
  bits = str(IMAGE_SHAPE[0]*IMAGE_SHAPE[1]-1)
  text = " module mem( \n input logic rst, \n output logic ["+bits+":0][2:0]mem \n); \n always_comb begin \n"
  for h in range(0, img.shape[0]):
    for w in range(0, img.shape[1]):
      blue = dec2bin(int((img[h,w,0]/255) * 15), 4)
      green = dec2bin(int((img[h,w,1]/255) * 15), 4)
      red = dec2bin(int((img[h,w,2]/255) * 15), 4)  
      text+=(write(red,green,blue,addr,h,w))
      addr += 1
  text += "end \nendmodule"
  fo.write(text);
encoding("mem.sv")

In [ ]:
exits = []
walls = []
for x_coordinate in range(IMAGE_SHAPE[0]):
  for y_coordinate in range(IMAGE_SHAPE[1]):
    if (img[x_coordinate, y_coordinate][1] == 255 ) : 
      exits.append([x_coordinate, y_coordinate])
    elif(img[x_coordinate, y_coordinate][0] == 255):
      walls.append([x_coordinate, y_coordinate])

def manhattan(current_pixel,exits) :
  distances = []  
  if(current_pixel in walls):
    temp_dist = 999
    distances.append(temp_dist)
  else : 
    for exit in exits:
      temp_dist = abs(current_pixel[0] - exit[0]) + abs(current_pixel[1] - exit[1])
      distances.append(temp_dist)
  return (min(distances))

weights = [[None for _ in range(IMAGE_SHAPE[0])]for _ in range(IMAGE_SHAPE[1]) ] 
for x in range(len(weights[0])):
  for y in range(len(weights[1])):
    dist = manhattan([x,y],exits)
    weights[x][y] = dist

In [ ]:
def display_weights():
    s = [[str(e) for e in row] for row in weights]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = ' '.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))
# display_weights()


In [ ]:
def write_weights(filename,weights) : 
  address = 0;
  bits = str(IMAGE_SHAPE[0]*IMAGE_SHAPE[1]-1)
  string = "module weights( \n input logic rst, \n output logic ["+bits+":0][12:0]weight \n); \n always_comb begin \n"  
  for x in range(len(weights[0])):
    for y in range(len(weights[1])):
      string += (("weight[%s] =" % address) + str(weights[x][y]) + ";" + '\n')
      address+=1
  string += "end \nendmodule"
  fo = open(filename, "w+")
  fo.write(string)
write_weights('weights.sv',weights)